<a href="https://colab.research.google.com/github/adasrinivas1229/MYPROJECTS/blob/master/Cganloss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install git+https://www.github.com/keras-team/keras-contrib.git
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import os
from google.colab import drive
from glob import glob
from keras import Input, Model
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, BatchNormalization, Activation
from keras.layers import Add, Conv2DTranspose, ZeroPadding2D, LeakyReLU
from keras.optimizers import Adam
from imageio import imread
from skimage.transform import resize
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

In [ ]:
def residual_block(x):

    """
    Residual block
    """
    
    res = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same")(x)
    res = BatchNormalization(axis = 3, momentum = 0.8, epsilon = 1e-5)(res)
    res = Activation('relu')(res)
    res = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same")(res)
    res = BatchNormalization(axis = 3, momentum = 0.8, epsilon = 1e-5)(res)
    
    return Add()([res, x])

In [ ]:
from keras.layers import Layer, InputSpec
from keras import initializers, regularizers, constraints
from keras import backend as K
#from tensorflow import backend as tf
class InstanceNormalization(Layer):
    """Instance normalization layer.
    Normalize the activations of the previous layer at each step,
    i.e. applies a transformation that maintains the mean activation
    close to 0 and the activation standard deviation close to 1.
    # Arguments
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `InstanceNormalization`.
            Setting `axis=None` will normalize all values in each
            instance of the batch.
            Axis 0 is the batch dimension. `axis` cannot be set to 0 to avoid errors.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a Sequential model.
    # Output shape
        Same shape as input.
    # References
        - [Layer Normalization](https://arxiv.org/abs/1607.06450)
        - [Instance Normalization: The Missing Ingredient for Fast Stylization](
        https://arxiv.org/abs/1607.08022)
    """
    def __init__(self,
                 axis=None,
                 epsilon=1e-3,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        ndim = len(input_shape)
        if self.axis == 0:
            raise ValueError('Axis cannot be zero')

        if (self.axis is not None) and (ndim == 2):
            raise ValueError('Cannot specify axis for rank 1 tensor')

        self.input_spec = InputSpec(ndim=ndim)

        if self.axis is None:
            shape = (1,)
        else:
            shape = (input_shape[self.axis],)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        reduction_axes = list(range(0, len(input_shape)))

        if self.axis is not None:
            del reduction_axes[self.axis]

        del reduction_axes[0]

        mean = K.mean(inputs, reduction_axes, keepdims=True)
        stddev = K.std(inputs, reduction_axes, keepdims=True) + self.epsilon
        normed = (inputs - mean) / stddev

        broadcast_shape = [1] * len(input_shape)
        if self.axis is not None:
            broadcast_shape[self.axis] = input_shape[self.axis]

        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            normed = normed * broadcast_gamma
        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            normed = normed + broadcast_beta
        return normed

    def get_config(self):
        config = {
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(InstanceNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [ ]:
def build_generator():

    """
    Creating a generator network with the hyperparameters defined below
    """
    
    input_shape = (128, 128, 3)
    residual_blocks = 6
    input_layer = Input(shape = input_shape)
    
    
    ## 1st Convolutional Block
    x = Conv2D(filters = 32, kernel_size = 7, strides = 1, padding = "same")(input_layer)
    x = InstanceNormalization(axis = 1)(x)
    x = Activation("relu")(x)
    
    ## 2nd Convolutional Block
    x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = "same")(x)
    x = InstanceNormalization(axis =1)(x)
    x = Activation("relu")(x)
    
    ## 3rd Convolutional Block
    x = Conv2D(filters = 128, kernel_size = 3, strides = 2, padding = "same")(x)
    x = InstanceNormalization(axis =1)(x)
    x = Activation("relu")(x)
    
    
    ## Residual blocks
    for _ in range(residual_blocks):
      x = residual_block(x)
      
    
    ## 1st Upsampling Block
    x = Conv2DTranspose(filters = 64, kernel_size = 3, strides = 2, padding = "same", 
                        use_bias = False)(x)
    x = InstanceNormalization(axis =1)(x)
    x = Activation("relu")(x)
    
    ## 2nd Upsampling Block
    x = Conv2DTranspose(filters = 32, kernel_size = 3, strides = 2, padding = "same", 
                        use_bias = False)(x)
    x = InstanceNormalization(axis =1)(x)
    x = Activation("relu")(x)
    
    ## Last Convolutional Layer
    x = Conv2D(filters = 3, kernel_size = 7, strides = 1, padding = "same")(x)
    output = Activation("tanh")(x)
    
    
    model = Model(inputs = [input_layer], outputs = [output])
    return model


In [ ]:
def build_discriminator():

    """
    Create a discriminator network using the hyperparameters defined below
    """
    input_shape = (128, 128, 3)
    hidden_layers = 3
    input_layer = Input(shape = input_shape) 
    x = ZeroPadding2D(padding = (1, 1))(input_layer)
    
    
    ## 1st Convolutional Block
    x = Conv2D(filters = 64, kernel_size = 4, strides = 2, padding = "valid")(x)
    x = LeakyReLU(alpha = 0.2)(x)
    x = ZeroPadding2D(padding = (1, 1))(x)
    
    ## 3 Hidden Convolutional Blocks
    for i in range(1, hidden_layers + 1):
      x = Conv2D(filters = 2 ** i * 64, kernel_size = 4, strides = 2, padding = "valid")(x) #128,256,512
      x = InstanceNormalization(axis =1)(x)
      x = LeakyReLU(alpha = 0.2)(x)
      x = ZeroPadding2D(padding = (1, 1))(x)
      
    
    ## Last Convolutional Layer
    output = Conv2D(filters = 1, kernel_size = 4, strides = 1, activation = "sigmoid")(x)
    
    model = Model(inputs = [input_layer], outputs = [output])
    return model


In [ ]:
def load_images(data_dir):
  
  imagesA = glob(data_dir + '/trainA/*.*')
  imagesB = glob(data_dir + '/trainB/*.*')
  
  allImagesA = []
  allImagesB = []
  
  for index, filename in enumerate(imagesA):
    imgA = imread(filename, pilmode = "RGB")
    imgB = imread(imagesB[index], pilmode = "RGB")
    
    imgA = resize(imgA, (128, 128))
    imgB = resize(imgB, (128, 128))
    
    if np.random.random() > 0.5:
      imgA = np.fliplr(imgA)
      imgB = np.fliplr(imgB)
      
    
    allImagesA.append(imgA)
    allImagesB.append(imgB)
    
  
  ## Normalize images
  allImagesA = np.array(allImagesA) / 127.5 - 1.
  allImagesB = np.array(allImagesB) / 127.5 - 1.
  
  return allImagesA, allImagesB

In [ ]:
def load_test_batch(data_dir, batch_size):
  
  imagesA = glob(data_dir + '/testA/*.*')
  imagesB = glob(data_dir + '/testB/*.*')
  
  imagesA = np.random.choice(a = imagesA, size = batch_size)
  imagesB = np.random.choice(a = imagesB, size = batch_size)
  
  allA = []
  allB = []
  
  for i in range(len(imagesA)):
    ## Load and resize images
    imgA = resize(imread(imagesA[i], pilmode = 'RGB').astype(np.float32), (128, 128))
    imgB = resize(imread(imagesB[i], pilmode = 'RGB').astype(np.float32), (128, 128))
    
    allA.append(imgA)
    allB.append(imgB)
  #normalize images as -1 ,1
  return np.array(allA) / 127.5 - 1.0, np.array(allB) / 127.5 - 1.0

In [ ]:
drive.mount('/content/drive')
data_dir = 'drive/My Drive/Colab Notebooks/monet2photo/'

batch_size = 1
epochs = 10
mode = 'train'

if mode == 'train':
  
  #Load dataset
  imagesA, imagesB = load_images(data_dir = data_dir)
  
  #Define the common optimizer
  common_optimizer = Adam(0.002, 0.5)
  ## Build and compile discriminator networks
  discriminatorA = build_discriminator()
  
  discriminatorB = build_discriminator()
  
  #Build generator networks
  generatorA_to_B = build_generator()
  generatorB_to_A = build_generator()
  #Create an adversarial network
  inputA = Input(shape = (128, 128, 3))
  inputB = Input(shape = (128, 128, 3))
    
  
  # --> Generated images using both of the generator networks
  generatedB = generatorA_to_B(inputA)
  generatedA = generatorB_to_A(inputB)
  print(generatedB)
  
  #--> Reconstruct the images back to the original ones
  reconstructedA = generatorB_to_A(generatedB)
  reconstructedB = generatorA_to_B(generatedA)
  print(reconstructedA)
  
 
  generatedA_Id = generatorB_to_A(inputA)
  generatedB_Id = generatorA_to_B(inputB)
  #print(generatedA_Id)
 
  #Make both of the discriminator networks non-trainable
  discriminatorA.trainable = False
  discriminatorB.trainable = False
    
  probsA = discriminatorA(generatedA)
  probsB = discriminatorB(generatedB)
  

  discriminatorA.compile(loss = 'mse', 
                           optimizer = common_optimizer,
                           metrics = ['accuracy'])
  discriminatorB.compile(loss = 'mse',
                           optimizer = common_optimizer,
                           metrics = ['accuracy'])
  
  
  adversarial_model = Model(inputs = [inputA, inputB],
                              outputs = [probsA, probsB, 
                                         reconstructedA, reconstructedB])
  """
  adversarial_model.compile(loss = ['mse', 'mse', 'mae', 'mae', 'mae', 'mae'],
                              loss_weights = [1, 1, 10.0, 10.0, 1.0, 1.0],
                            optimizer = common_optimizer)
  """   
  adversarial_model.compile(loss = ['mse', 'mse', 'mae', 'mae'],
                              loss_weights = [1, 1, 10.0, 10.0],
                              optimizer = common_optimizer)
  real_labels = np.ones((batch_size, 7, 7, 1))
  fake_labels = np.zeros((batch_size, 7, 7, 1))
    
  for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    
    num_batches = int(min(imagesA.shape[0], imagesB.shape[0]) / batch_size)
    print("Number of batches: {}".format(num_batches))

  for index in range(num_batches):
    print("Batch: {}".format(index))
        
    ## Sample images
    batchA = imagesA[index * batch_size: (index + 1) * batch_size]
    batchB = imagesB[index * batch_size: (index + 1) * batch_size]
        
    ## Translate images to opposite domain
    generatedB = generatorA_to_B.predict(batchA)
    generatedA = generatorB_to_A.predict(batchB)
        
    ## Train the discriminator A on real and fake images
    D_A_Loss1 = discriminatorA.train_on_batch(batchA, real_labels) #x-batchA
    D_A_Loss2 = discriminatorA.train_on_batch(generatedA, fake_labels)#y-generatedA
        
    ## Train the discriminator B on real and fake images
    D_B_Loss1 = discriminatorB.train_on_batch(batchB, real_labels)#x^
    D_B_Loss2 = discriminatorB.train_on_batch(generatedB, fake_labels)#generatedB-y^
        
    ## Calculate the total discriminator loss
    D_loss = 0.5 * np.add(0.5 * np.add(D_A_Loss1, D_A_Loss2), 
                              0.5 * np.add(D_B_Loss1, D_B_Loss2))
        
    print("D_Loss: {}".format(D_loss))
        
        
    #Train the generator networks   
    G_loss = adversarial_model.train_on_batch([batchA, batchB],
                                                  [real_labels, real_labels,
                                                   batchA, batchB])
        
    print("G_Loss: {}".format(G_loss))
    
    
      
    # Sample and save images after every 10 epochs
    if epoch % 10 == 0:
      # Get a batch of test data
      batchA, batchB = load_test_batch(data_dir = data_dir, batch_size = 2)
        
      # Generate images
      generatedB = generatorA_to_B.predict(batchA)
      generatedA = generatorB_to_A.predict(batchB)
      print(generatedB)
      # Get reconstructed images
      recons_A = generatorB_to_A.predict(generatedB)
      recons_B = generatorA_to_B.predict(generatedA)
      print(recons_A)
 


        


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tensor("model_49/activation_252/Tanh:0", shape=(None, 128, None, 3), dtype=float32)
Tensor("model_50_1/activation_264/Tanh:0", shape=(None, 128, None, 3), dtype=float32)
Epoch: 0
Number of batches: 100
Epoch: 1
Number of batches: 100
Epoch: 2
Number of batches: 100
Epoch: 3
Number of batches: 100
Epoch: 4
Number of batches: 100
Epoch: 5
Number of batches: 100
Epoch: 6
Number of batches: 100
Epoch: 7
Number of batches: 100
Epoch: 8
Number of batches: 100
Epoch: 9
Number of batches: 100
Batch: 0
D_Loss: [0.44610944 0.43367347]
G_Loss: [21.364355, 0.0926803, 0.5561995, 1.0161757, 1.0553718]
Batch: 1
D_Loss: [0.30693766 0.5408163 ]
G_Loss: [4.4461637, 0.2501974, 0.681328, 0.2125904, 0.13887341]
Batch: 2
D_Loss: [0.31844535 0.5       ]
G_Loss: [1.0089115, 0.21001205, 0.6918007, 0.004920561, 0.005789315]
Batch: 3
D_Loss: [0.31845784 0.5       ]
G_Loss: [0.96485627,